In [1]:
# Устанавливаем все необходимые библиотеки для работы ноутбука
%pip install --upgrade pip

# Базовые числовые и научные пакеты
%pip install numpy opencv-python matplotlib scipy scikit-learn pillow

# PyTorch (CPU-только)
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

# OCR и эмбеддинги
%pip install easyocr sentence-transformers

# Whisper от OpenAI
%pip install openai-whisper

# Установка транслитератора
%pip install Unidecode

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


При необходимости перезапустите ядро (Kernel → Restart) и только потом выполняйте остальные ячейки с импортами и вашим кодом. Так вы гарантированно избежите «ModuleNotFoundError» и подобных ошибок на чистой системе.

In [2]:
import cv2
import easyocr
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from scipy.stats import norm
import unidecode from unidecode

c:\Users\Илья\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return super().default(obj)

In [4]:
def make_safe_name(name: str) -> str:
    """
    1) Транслитерирует Юникод в ASCII
    2) Заменяет пробелы на '_'
    3) Удаляет из строки всё, кроме A–Z, a–z, 0–9, '_' и '-'
    """
    ascii_name = unidecode(name)            # e.g. "Akadem. gramotnost' - W6_L3"
    ascii_name = ascii_name.replace(" ", "_")# "Akadem._gramotnost'_-_W6_L3"
    # удаляем точки, апострофы и прочие посторонние символы
    safe = re.sub(r"[^A-Za-z0-9_\-]", "", ascii_name)
    return safe

def text_quantity(text):
    return len(text.split())

def frame_diff(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    return np.sum(cv2.absdiff(gray1, gray2)) / gray1.size

def normalize_list(values):
    min_v, max_v = min(values), max(values)
    return [(v - min_v) / (max_v - min_v) if max_v > min_v else 0 for v in values]

def standardize_list(values):
    mean_val = np.mean(values)
    std_val = np.std(values)
    if std_val == 0:
        return [0.5 for _ in values]
    z_scores = [(v - mean_val) / std_val for v in values]
    return [norm.cdf(z) for z in z_scores]

def calculate_recency(candidates, selected_frames, interval_seconds):
    # Вычисляем recency для каждого кандидата по хронологии
    selected_timecodes = set(f['timecode'] for f in selected_frames)
    sorted_candidates = sorted(candidates, key=lambda f: f['timecode'])
    recency_vals = []
    counter = 0
    for frame in sorted_candidates:
        if frame['timecode'] in selected_timecodes:
            recency_vals.append(0.0)
            counter = 0
        else:
            counter += 1
            val = 0.1 + 0.2 * (counter - 1)
            if val > 1.0:
                val = 1.0
            recency_vals.append(val)
    return recency_vals

def extract_key_frames(
    video_path: str,
    all_frames_dir: str,
    top_frames_dir: str,
    coords_pct: dict,
    weight_text: float,
    weight_diff: float,
    weight_similarity: float,
    weight_recency: float,
    interval_seconds: float = 10.0,
    top_n: int = 5,
    max_iter: int = 100
):
    """
    Извлечение ключевых кадров из видео с итеративным учётом recency.
    
    Параметры:
      video_path          – путь к видеофайлу;
      all_frames_dir      – папка для всех кадров и метрик;
      top_frames_dir      – папка для сохранения итоговых top_n кадров;
      coords_pct          – словарь процентных координат {'x1', 'y1', 'x2', 'y2'} в диапазоне [0,1];
      weight_text         – вес текстовой метрики;
      weight_diff         – вес diff-метрики;
      weight_similarity   – вес семантической метрики;
      weight_recency      – вес recency-метрики;
      interval_seconds    – интервал между анализируемыми кадрами (сек);
      top_n               – число кадров в итоговом наборе;
      max_iter            – макс. число итераций для уточнения выбора.
    """
    import os
    import cv2
    import json
    import numpy as np
    import easyocr
    from scipy.stats import norm
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import matplotlib.pyplot as plt

    # 1) Инициализация OCR и эмбеддера
    reader = easyocr.Reader(['en', 'ru'], gpu=True)
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    emb_dim = embedder.get_sentence_embedding_dimension()

    # 2) Открываем видео и узнаём базовые величины
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS) or 1.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    duration = total_frames / fps if fps > 0 else 0.0

    # 3) Читаем первый кадр для размеров
    ret, frame0 = cap.read()
    if not ret:
        print(f"[ERROR] Не удалось открыть видео {video_path}")
        cap.release()
        return
    h, w = frame0.shape[:2]
    x1 = int(coords_pct["x1"] * w);  y1 = int(coords_pct["y1"] * h)
    x2 = int(coords_pct["x2"] * w);  y2 = int(coords_pct["y2"] * h)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # 4) Собираем кандидатов
    candidates = []
    prev_frames = []
    all_embs    = []
    frame_idx   = 0

    while True:
        ret, frame = cap.read()
        if not ret or frame_idx / fps >= duration - interval_seconds:
            break

        if frame_idx % int(fps * interval_seconds) == 0 and frame_idx != 0:
            crop = frame[y1:y2, x1:x2]

            # текстовая метрика
            txts = reader.readtext(crop)
            text = " ".join([t[1] for t in txts])
            t_qty = len(text.split())

            # diff-метрика
            if prev_frames:
                gray_new = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                gray_old = cv2.cvtColor(prev_frames[-1], cv2.COLOR_BGR2GRAY)
                t_diff = float(np.sum(cv2.absdiff(gray_new, gray_old))) / gray_new.size
            else:
                t_diff = 0.0

            # семантическая уникальность
            if text.strip():
                emb = embedder.encode(text)
            else:
                emb = np.zeros(emb_dim, dtype=float)
            if all_embs:
                sims = [cosine_similarity([emb], [e])[0][0] for e in all_embs]
                t_sim = 1.0 - max(sims)
            else:
                t_sim = 1.0

            candidates.append({
                'frame':      crop.copy(),
                'timecode':   frame_idx / fps,
                'text_qty':   t_qty,
                'diff_score': t_diff,
                'sim_score':  t_sim,
                'rec_norm':   0.0
            })
            prev_frames.append(crop.copy())
            all_embs.append(emb)

        frame_idx += 1

    cap.release()

    # 5) Нормализация метрик
    def normalize(vs):
        arr = np.array(vs, float)
        return (arr - arr.min()) / (arr.max() - arr.min()) if arr.max() > arr.min() else np.ones_like(arr)

    text_norm = normalize([c['text_qty']   for c in candidates])
    diff_norm = normalize([c['diff_score'] for c in candidates])
    sim_norm  = normalize([c['sim_score']  for c in candidates])

    for i, c in enumerate(candidates):
        c['text_n'] = text_norm[i]
        c['diff_n'] = diff_norm[i]
        c['sim_n']  = sim_norm[i]

    # 6) Начальный отбор
    selected = sorted(
        candidates,
        key=lambda c: (weight_text * c['text_n'] +
                       weight_diff * c['diff_n'] +
                       weight_similarity * c['sim_n']),
        reverse=True
    )[:top_n]

    # 7) Помощник для recency
    def calc_recency(cands, sel):
        times_sel = {int(round(f['timecode'])) for f in sel}
        sorted_c = sorted(cands, key=lambda x: x['timecode'])
        recs, cnt = [], 0
        for f in sorted_c:
            t = int(round(f['timecode']))
            if t in times_sel:
                recs.append(0.0); cnt = 0
            else:
                cnt += 1
                recs.append(min(1.0, 0.1 + 0.2*(cnt-1)))
        return recs

    # 8) Итеративное уточнение composite-score
    for _ in range(max_iter):
        prev_set = {int(round(f['timecode'])) for f in selected}
        recs = calc_recency(candidates, selected)
        for i, c in enumerate(candidates):
            c['rec_norm'] = recs[i]
            c['comp'] = (
                weight_text       * c['text_n'] +
                weight_diff       * c['diff_n'] +
                weight_similarity * c['sim_n'] +
                weight_recency    * c['rec_norm']
            )
        selected = sorted(candidates, key=lambda x: x['comp'], reverse=True)[:top_n]
        if prev_set == {int(round(f['timecode'])) for f in selected}:
            break

    # 9) (опционально) график
    times = [c['timecode'] for c in candidates]
    w = interval_seconds * 0.8
    plt.figure(figsize=(12,6))
    bottom = np.zeros(len(times))
    for wt, arr, lbl in [
      (weight_text, text_norm, 'Text'),
      (weight_diff, diff_norm, 'Diff'),
      (weight_similarity, sim_norm, 'Sim'),
      (weight_recency, [c['rec_norm'] for c in candidates], 'Rec')
    ]:
        vals = wt * np.array(arr)
        plt.bar(times, vals, width=w, bottom=bottom, label=lbl)
        bottom += vals
    plt.xlabel("Time (s)")
    plt.ylabel("Score components")
    plt.legend(); plt.grid(True)
    os.makedirs(all_frames_dir, exist_ok=True)
    plt.savefig(os.path.join(all_frames_dir, "composite_score_components.png"))
    plt.close()

    # 10) Сохранение всех кадров
    os.makedirs(all_frames_dir, exist_ok=True)
    all_data = []
    for c in candidates:
        tc = int(round(c['timecode']))
        fname = f"{tc:06d}.jpg"
        cv2.imwrite(os.path.join(all_frames_dir, fname), c['frame'])
        info = {k: v for k, v in c.items() if k != 'frame'}
        info['filename'] = fname
        all_data.append(info)
    with open(os.path.join(all_frames_dir, "all_frames_data.json"), "w", encoding="utf-8") as jf:
        json.dump(all_data, jf, cls=NumpyEncoder, ensure_ascii=False, indent=4)

    # 11) Сохранение top-n
    os.makedirs(top_frames_dir, exist_ok=True)
    top_data = []
    for idx, c in enumerate(selected, start=1):
        tc = int(round(c['timecode']))
        fname = f"{tc:06d}s_top_{idx}.jpg"
        cv2.imwrite(os.path.join(top_frames_dir, fname), c['frame'])
        info = {k: v for k, v in c.items() if k != 'frame'}
        info['filename'] = fname
        top_data.append(info)
    with open(os.path.join(top_frames_dir, "top_frames_data.json"), "w", encoding="utf-8") as jf:
        json.dump(top_data, jf, cls=NumpyEncoder, ensure_ascii=False, indent=4)

    print(f"\nОбработано {len(candidates)} кадров → {all_frames_dir}")
    print(f"Сохранено top-{top_n} кадров → {top_frames_dir}")



In [5]:
import os, json
from pathlib import Path

# ——— Определяем местоположение ноутбука — обычно это рабочая папка Jupyter
notebook_dir = Path().resolve()
config_path  = notebook_dir / "config.json"

# ——— Если нет — создаём с шаблоном (с процентными координатами)
if not config_path.exists():
    default_config = {
      "base_directory": str(notebook_dir),
      "number_video_images": 10,
      "weights": {
        "WEIGHT_TEXT":       0.1,
        "WEIGHT_DIFF":       0.6,
        "WEIGHT_SIMILARITY": 0.2,
        "WEIGHT_RECENCY":    0.05
      },
      "coordinates_pct": {
        "x1": 0.0,   # лево
        "y1": 0.0,   # верх
        "x2": 1.0,   # право
        "y2": 1.0    # низ
      }
    }
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(default_config, f, cls=NumpyEncoder, ensure_ascii=False, indent=4)
    print(f"[INFO] Создан файл конфигурации: {config_path}")
else:
    print(f"[INFO] Загружаем конфигурацию из {config_path}")

# ——— Читаем конфиг
with open(config_path, "r", encoding="utf-8") as f:
    cfg = json.load(f)



[INFO] Загружаем конфигурацию из D:\local\config.json


In [7]:
# ——— Подтягиваем все параметры
BASE_DIR             = Path(cfg["base_directory"])
NUMBER_VIDEO_IMAGES  = cfg["number_video_images"]
WEIGHT_TEXT          = cfg["weights"]["WEIGHT_TEXT"]
WEIGHT_DIFF          = cfg["weights"]["WEIGHT_DIFF"]
WEIGHT_SIMILARITY    = cfg["weights"]["WEIGHT_SIMILARITY"]
WEIGHT_RECENCY       = cfg["weights"]["WEIGHT_RECENCY"]

# ——— Процентные координаты (0.0–1.0)
coords_pct = cfg["coordinates_pct"]
# сами пиксели будем вычислять уже внутрь функции, когда знаем ширину/высоту кадра

# ——— Формируем нужные папки внутри BASE_DIR
video_directory             = BASE_DIR / "input"
all_frames_output_directory = BASE_DIR / "work"  / "all_images"
top_frames_output_directory = BASE_DIR / "output"/ "images"

# ——— Создадим их, если ещё нет
os.makedirs(video_directory,             exist_ok=True)
os.makedirs(all_frames_output_directory, exist_ok=True)
os.makedirs(top_frames_output_directory, exist_ok=True)


In [ ]:
for video_file in os.listdir(video_directory):
    if video_file.lower().endswith(('.mp4', '.avi', '.mkv')):
        video_path = Path(video_directory) / video_file
        raw_name   = video_path.stem
        video_name = make_safe_name(raw_name)

        video_all_frames_dir = Path(all_frames_output_directory) / video_name
        video_top_frames_dir = Path(top_frames_output_directory) / video_name
        os.makedirs(video_all_frames_dir, exist_ok=True)
        os.makedirs(video_top_frames_dir, exist_ok=True)

        extract_key_frames(
            str(video_path),
            str(video_all_frames_dir),
            str(video_top_frames_dir),
            coords_pct,
            WEIGHT_TEXT,
            WEIGHT_DIFF,
            WEIGHT_SIMILARITY,
            WEIGHT_RECENCY,
            interval_seconds=10,
            top_n=NUMBER_VIDEO_IMAGES,
            max_iter=100
        )


In [8]:
import os
import re
import json
import whisper

import subprocess


In [9]:
def process_videos_transcription(video_directory, whisper_json_directory, whisper_text_directory):
    """
    Функция транскрипции:
      1. Для каждого видео в video_directory, если итоговый TXT уже существует – пропускаем.
      2. Если TXT отсутствует, проверяется наличие JSON:
         - Если JSON существует, он загружается.
         - Если JSON отсутствует, запускается транскрипция с Whisper (base) и результат сохраняется.
      3. Итоговый текст формируется как объединение сегментов (без вставки меток) и сохраняется в TXT.
    """
    print("[INFO] Загружаем модель Whisper (base)...")
    model = whisper.load_model("base")
    
    for video_file in os.listdir(video_directory):
        if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue
        
        video_path = os.path.join(video_directory, video_file)
        video_name_no_ext = os.path.splitext(video_file)[0]
        video_name_clean = video_name_no_ext.replace(" ", "_")
        
        print(f"[INFO] Обработка видео: {video_path}")
        
        json_output_path = os.path.join(whisper_json_directory, video_name_clean + ".json")
        txt_output_path = os.path.join(whisper_text_directory, video_name_clean + ".txt")
        
        # Если итоговый TXT уже существует, пропускаем обработку этого видео
        if os.path.exists(txt_output_path):
            print(f"[INFO] TXT файл для видео '{video_name_no_ext}' уже существует ({txt_output_path}). Пропускаем обработку.")
            continue
        
        # Если JSON существует, загружаем его, иначе запускаем транскрипцию
        if os.path.exists(json_output_path):
            print(f"[INFO] JSON транскрипция для видео '{video_name_no_ext}' уже существует. Загружаем JSON.")
            with open(json_output_path, "r", encoding="utf-8") as f:
                transcription = json.load(f)
        else:
            print(f"[INFO] JSON транскрипция для видео '{video_name_no_ext}' не найдена. Запускаем транскрипцию.")
            transcription = model.transcribe(video_path, task="transcribe")
            with open(json_output_path, "w", encoding="utf-8") as f:
                json.dump(transcription, f, cls=NumpyEncoder, ensure_ascii=False, indent=4)
            print(f"[INFO] JSON транскрипция сохранена: {json_output_path}")
        
        # Формируем итоговый текст транскрипции как объединение сегментов (без маркеров)
        if "segments" in transcription:
            transcription["text"] = " ".join(seg["text"] for seg in transcription["segments"])
            print(f"[DEBUG] Итоговый текст сформирован. Длина текста: {len(transcription['text'])} символов")
        else:
            print(f"[WARN] Нет сегментов в транскрипции для видео '{video_name_no_ext}'")
        
        # Сохраняем итоговый текст в TXT
        with open(txt_output_path, "w", encoding="utf-8") as f:
            f.write(transcription.get("text", ""))
        print(f"[INFO] TXT файл сохранён: {txt_output_path}")  

In [10]:
# Каталоги для транскрипции на основе BASE_DIR из config.json
video_directory         = BASE_DIR / "input"
whisper_json_directory  = BASE_DIR / "work"  / "whisper_json"
whisper_text_directory  = BASE_DIR / "work"  / "whisper_text"
extracted_frames_base   = BASE_DIR / "output"/ "images"

# Создаём директории, если ещё нет
os.makedirs(whisper_json_directory, exist_ok=True)
os.makedirs(whisper_text_directory, exist_ok=True)

# Запускаем транскрипцию
process_videos_transcription(
    str(video_directory),
    str(whisper_json_directory),
    str(whisper_text_directory)
)



[INFO] Загружаем модель Whisper (base)...
[INFO] Обработка видео: D:\local\input\W1_L1 (DL_1_1).mp4
[INFO] JSON транскрипция для видео 'W1_L1 (DL_1_1)' не найдена. Запускаем транскрипцию.
[INFO] JSON транскрипция сохранена: D:\local\work\whisper_json\W1_L1_(DL_1_1).json
[DEBUG] Итоговый текст сформирован. Длина текста: 4599 символов
[INFO] TXT файл сохранён: D:\local\work\whisper_text\W1_L1_(DL_1_1).txt
[INFO] Обработка видео: D:\local\input\W1_L12(DL_1_12).mp4
[INFO] JSON транскрипция для видео 'W1_L12(DL_1_12)' не найдена. Запускаем транскрипцию.
[INFO] JSON транскрипция сохранена: D:\local\work\whisper_json\W1_L12(DL_1_12).json
[DEBUG] Итоговый текст сформирован. Длина текста: 10491 символов
[INFO] TXT файл сохранён: D:\local\work\whisper_text\W1_L12(DL_1_12).txt
[INFO] Обработка видео: D:\local\input\W2_L12 (DL_2_12).mp4
[INFO] JSON транскрипция для видео 'W2_L12 (DL_2_12)' не найдена. Запускаем транскрипцию.
[INFO] JSON транскрипция сохранена: D:\local\work\whisper_json\W2_L12_(DL